In [19]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd 
import keras
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import tensorflow as tf
import re

from sklearn.model_selection import KFold


#print how many rows and columns are there in the csv file
df = pd.read_csv('train-balanced-sarcasm.csv')


df_useful_x = df[['comment','label']]
df_useful_y = df[['label']]
df_useful_x['comment'] = df_useful_x['comment'].astype('str')

df_useful_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 2 columns):
comment    1010826 non-null object
label      1010826 non-null int64
dtypes: int64(1), object(1)
memory usage: 15.4+ MB


C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
df_useful_x
df_useful_x['pre_clean_len'] = [len(t) for t in df_useful_x.comment]
df_useful_x


C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,comment,label,pre_clean_len
0,NC and NH.,0,10
1,You do know west teams play against west teams...,0,74
2,"They were underdogs earlier today, but since G...",0,121
3,"This meme isn't funny none of the ""new york ni...",0,60
4,I could use one of those tools.,0,31
...,...,...,...
1010821,I'm sure that Iran and N. Korea have the techn...,1,92
1010822,"whatever you do, don't vote green!",1,34
1010823,Perhaps this is an atheist conspiracy to make ...,1,66
1010824,The Slavs got their own country - it is called...,1,53


In [22]:
#Ahad
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
    #word=[w for w in text if w not in not in stop]
    

df_useful_x['comment']=df_useful_x['comment'].apply(stopwords)
df_useful_x

[nltk_data] Downloading package stopwords to C:\Users\Abdul Ahad
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,comment,label,pre_clean_len
0,NC NH.,0,10
1,You know west teams play west teams east teams...,0,74
2,"They underdogs earlier today, since Gronk's an...",0,121
3,"This meme funny none ""new york nigga"" ones are.",0,60
4,I could use one tools.,0,31
...,...,...,...
1010821,I'm sure Iran N. Korea technology create pig/b...,1,92
1010822,"whatever do, vote green!",1,34
1010823,Perhaps atheist conspiracy make Christians loo...,1,66
1010824,The Slavs got country - called Kosovo,1,53


In [23]:
df_useful_x

,comment,label,pre_clean_len
0,NC NH.,0,10
1,You know west teams play west teams east teams...,0,74
2,"They underdogs earlier today, since Gronk's an...",0,121
3,"This meme funny none ""new york nigga"" ones are.",0,60
4,I could use one tools.,0,31
...,...,...,...
1010821,I'm sure Iran N. Korea technology create pig/b...,1,92
1010822,"whatever do, vote green!",1,34
1010823,Perhaps atheist conspiracy make Christians loo...,1,66
1010824,The Slavs got country - called Kosovo,1,53


In [24]:
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def text_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()



#nums = [0,1010826]
nums = [0,1010826]
print ("Cleaning and parsing the comments...\n")
clean_comment_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%10000 == 0 ):
        print ("comments %d of %d has been processed" % ( i+1, nums[1] ))                                                                    
    clean_comment_texts.append(text_cleaner(df_useful_x['comment'][i]))
    
clean_df = pd.DataFrame(clean_comment_texts,columns=['comment'])

clean_df.head()


Cleaning and parsing the comments...

comments 10000 of 1010826 has been processed


C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'.....'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


comments 20000 of 1010826 has been processed


C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'/'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'....'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


comments 30000 of 1010826 has been processed
comments 40000 of 1010826 has been processed
comments 50000 of 1010826 has been processed
comments 60000 of 1010826 has been processed


...

C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://localhost:88/emblem-bot/index.php" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


comments 860000 of 1010826 has been processed
comments 870000 of 1010826 has been processed
comments 880000 of 1010826 has been processed
comments 890000 of 1010826 has been processed
comments 900000 of 1010826 has been processed
comments 910000 of 1010826 has been processed
comments 920000 of 1010826 has been processed
comments 930000 of 1010826 has been processed


C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://last.word/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


comments 940000 of 1010826 has been processed
comments 950000 of 1010826 has been processed
comments 960000 of 1010826 has been processed
comments 970000 of 1010826 has been processed
comments 980000 of 1010826 has been processed
comments 990000 of 1010826 has been processed
comments 1000000 of 1010826 has been processed
comments 1010000 of 1010826 has been processed


,comment
0,nc nh
1,you know west teams play west teams east teams...
2,they underdogs earlier today since gronk s ann...
3,this meme funny none new york nigga ones are
4,i could use one tools


In [25]:
#clean_df.to_csv('cleandf.csv')
#clean_df = pd.read_csv('cleandf.csv')
#clean_df.columns
#clean_df=clean_df[['comment']]
clean_df


,comment
0,nc nh
1,you know west teams play west teams east teams...
2,they underdogs earlier today since gronk s ann...
3,this meme funny none new york nigga ones are
4,i could use one tools
...,...
1010821,i m sure iran n korea technology create pig bi...
1010822,whatever do vote green
1010823,perhaps atheist conspiracy make christians loo...
1010824,the slavs got country called kosovo


In [26]:
max_features= 1000
Reddit_Comments = clean_df['comment'].values
Reddit_Comments

array(['nc nh', 'you know west teams play west teams east teams right',
       'they underdogs earlier today since gronk s announcement afternoon vegas line moved patriots',
       ..., 'perhaps atheist conspiracy make christians look bad',
       'the slavs got country called kosovo',
       'values capitalism good money imprisoning people'], dtype=object)

In [27]:
import nltk
from keras.preprocessing.text import Tokenizer
# create the tokenizer
t = Tokenizer(num_words=500)
# fit the tokenizer on the documents
t.fit_on_texts(Reddit_Comments)
#token=[nltk.word_tokenize(comment) for comment in Reddit_Comments]

In [11]:
print(t.word_counts)
print(t.document_count)



OrderedDict([('nc', 170), ('nh', 28), ('you', 44766), ('know', 33036), ('west', 1021), ('team', 7960), ('play', 14588), ('east', 957), ('right', 24526), ('they', 23215), ('underdogs', 11), ('early', 1376), ('today', 2195), ('since', 6583), ('gronk', 81), ('s', 106990), ('announcement', 184), ('afternoon', 89), ('vegas', 222), ('line', 3011), ('move', 3636), ('patriot', 221), ('this', 22012), ('meme', 1381), ('funny', 3979), ('none', 1111), ('new', 9944), ('york', 417), ('nigga', 174), ('one', 33917), ('are', 5164), ('i', 220231), ('could', 15456), ('use', 19687), ('tools', 77), ('pay', 6905), ('attention', 1111), ('her', 2136), ('long', 6462), ('legal', 1089), ('kick', 1273), ('bed', 598), ('before', 1924), ('take', 16000), ('load', 871), ('trick', 571), ('treat', 950), ('general', 1134), ('weird', 1695), ('blade', 314), ('mastery', 97), ('masamune', 5), ('gtfo', 243), ('to', 6103), ('good', 29012), ('build', 2832), ('buy', 6795), ('game', 20572), ('save', 2748), ('would', 36008), ('lo

In [28]:
sequences = t.texts_to_sequences(Reddit_Comments)

TypeError: sequences_to_matrix() missing 1 required positional argument: 'sequences'

In [29]:
sequences

[[],
 [6, 17, 98, 21],
 [22, 446, 136, 2, 408],
 [23, 260, 80, 394, 183],
 [1, 37, 68, 14],
 [1, 242, 453, 156, 1, 432],
 [],
 [],
 [6, 149, 20, 196, 167],
 [1, 9, 85, 32],
 [1, 19, 9, 8],
 [187, 1, 124, 103],
 [22],
 [67, 225],
 [108, 301, 288],
 [35],
 [],
 [26, 220],
 [104],
 [],
 [237, 69, 374],
 [273, 148, 22, 27, 1, 37, 421],
 [28, 60, 59, 136],
 [84, 485],
 [177, 126],
 [],
 [401, 470],
 [23, 9, 31],
 [273, 79, 1, 86, 304, 4, 11],
 [23, 82, 52, 2, 45, 21],
 [80],
 [38, 22, 27, 103, 403, 128],
 [371, 114, 67],
 [13, 22, 61],
 [],
 [263],
 [42],
 [171, 1, 64, 136, 1],
 [1, 79, 1, 29, 146],
 [89, 4],
 [260, 39, 491],
 [43, 138, 15],
 [],
 [20],
 [122, 87, 322, 425, 9, 35],
 [13, 106, 45, 37, 353, 17, 325],
 [70, 116],
 [94, 1, 12, 1, 102, 8, 17, 66, 389],
 [179, 247, 374],
 [270, 270],
 [2],
 [],
 [214, 104, 129],
 [350, 43],
 [30, 144],
 [1, 198, 196, 422],
 [340, 401],
 [50, 59, 476],
 [113, 9, 195, 95, 19, 66, 8],
 [8, 129, 49, 180, 229, 442],
 [1, 198, 238],
 [1, 12, 401, 2, 40

In [30]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
top_words = 500
max_words = 20

from sklearn.model_selection import train_test_split
x = sequence.pad_sequences(sequences, maxlen=max_words)
y=df_useful_y.to_numpy()
X_train,X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [31]:
X_train.shape

(808660, 20)

In [32]:
X_test.shape

(202166, 20)

In [33]:
Y_train.shape

(808660, 1)

In [34]:
Y_test.shape

(202166, 1)

In [35]:
X_train = X_train[1:200]
Y_train = Y_train[1:200]

X_test = X_test[1:200]
Y_test = Y_test[1:200]

# pad upto max_words
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [36]:
X_train.shape

(199, 20)

In [37]:
model = Sequential()
model.add(Embedding(top_words, 8, input_length=max_words))
model.add(Conv1D(filters=16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             4000      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 20, 16)            400       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10, 16)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               40250     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 44,901
Trainable params: 44,901
Non-trainable params: 0
__________________________________________________

In [38]:
# Fit the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=128, verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

C:\Users\Abdul Ahad Khan\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 199 samples, validate on 199 samples
Epoch 1/10
199/199 [==============================] - 0s 713us/step - loss: 0.6927 - accuracy: 0.5276 - val_loss: 0.6909 - val_accuracy: 0.5528
Epoch 2/10
199/199 [==============================] - 0s 35us/step - loss: 0.6909 - accuracy: 0.5226 - val_loss: 0.6898 - val_accuracy: 0.5528
Epoch 3/10
199/199 [==============================] - 0s 40us/step - loss: 0.6895 - accuracy: 0.5226 - val_loss: 0.6894 - val_accuracy: 0.5528
Epoch 4/10
199/199 [==============================] - 0s 40us/step - loss: 0.6882 - accuracy: 0.5226 - val_loss: 0.6889 - val_accuracy: 0.5528
Epoch 5/10
199/199 [==============================] - 0s 40us/step - loss: 0.6867 - accuracy: 0.5276 - val_loss: 0.6886 - val_accuracy: 0.5528
Epoch 6/10
199/199 [==============================] - 0s 40us/step - loss: 0.6849 - accuracy: 0.5276 - val_loss: 0.6884 - val_accuracy: 0.5528
Epoch 7/10
199/199 [==============================] - 0s 45us/step - loss: 0.6826 - accuracy: 0